## Setup

You will need to make a copy of this notebook in your Google Drive before you can edit the homework files. You can do so with **File &rarr; Save a copy in Drive**.

In [1]:
%cd /gpfs/data/oermannlab/users/bml398/homework_fall2023/hw1/

/gpfs/data/oermannlab/users/bml398/homework_fall2023/hw1


In [2]:
import sys
print(sys.executable)

/gpfs/data/oermannlab/users/bml398/.conda/envs/cs_285/bin/python3.11


In [6]:
# #@title set up virtual display

# from pyvirtualdisplay import Display

# display = Display(visible=0, size=(1400, 900))
# display.start()

import os
os.environ["MUJOCO_GL"] = "egl"   # use GPU headless rendering

In [14]:
import sys

# Path you want to block
BAD = "/gpfs/share/apps/miniconda3/gpu/4.9.2/lib/python3.8/site-packages"

# Remove it from sys.path if present
sys.path = [p for p in sys.path if p != BAD]

# If an old/bad version of cffi already got imported, drop it
for mod in list(sys.modules):
    if mod.startswith("cffi") or mod == "_cffi_backend":
        sys.modules.pop(mod, None)

# Optionally, ensure your preferred user site is first
import site
user_site = site.getusersitepackages()
if user_site in sys.path:
    sys.path.remove(user_site)
sys.path.insert(0, user_site)

In [15]:
import gymnasium
from cs285.infrastructure.colab_utils import (
    wrap_env,
    show_video
)

env = wrap_env(gymnasium.make("Ant-v4", render_mode='rgb_array'))

observation = env.reset()
for i in range(100):
    env.render()
    action = env.action_space.sample()
    # print(f"action: {action}")
    obs, rew, term, truncated, info = env.step(action)
    if term:
      break;

env.close()
print('Loading video...')
show_video()

/gpfs/data/oermannlab/users/bml398/.conda/envs/cs_285/lib/python3.11/site-packages/gymnasium/envs/registration.py:512: DeprecationWarning: WARN: The environment Ant-v4 is out of date. You should consider upgrading to version `v5`.
  logger.deprecation(


Loading video...


## Editing Code

To edit code, click the folder icon on the left menu. Navigate to the corresponding file (`cs285_f2020/...`). Double click a file to open an editor. You will need to edit code in the following files:
```markdown
* cs285/policies/MLP_policy.py
* cs285/infrastructure/utils.py
* cs285/scripts/run_hw1.py
```

## Run Behavior Cloning (Problem 1)

Note that there is a timeout of about ~12 hours with Colab while it is active (and less if you close your browser window). We sync your edits to Google Drive so that you won't lose your work in the event of an instance timeout, but you will need to re-mount your Google Drive and re-install packages with every new instance.

In [16]:
#@title imports

import time

from cs285.scripts.run_hw1 import run_training_loop

# %load_ext autoreload
%reload_ext autoreload
%autoreload 2

In [17]:
#@title runtime arguments

class Args:

  def __getitem__(self, key):
    return getattr(self, key)

  def __setitem__(self, key, val):
    setattr(self, key, val)

  #@markdown expert data
  expert_policy_file = 'cs285/policies/experts/Ant.pkl' #@param
  expert_data = 'cs285/expert_data/expert_data_Ant-v4.pkl' #@param
  env_name = 'Ant-v4' #@param ['Ant-v4', 'Walker2d-v4', 'HalfCheetah-v4', 'Hopper-v4']
  exp_name = 'dagger_bc_ant' #@param
  do_dagger = True #@param {type: "boolean"}
  ep_len = 1000 #@param {type: "integer"}
  save_params = False #@param {type: "boolean"}

  num_agent_train_steps_per_iter = 1000 #@param {type: "integer"})
  n_iter = 10 #@param {type: "integer"})

  #@markdown batches & buffers
  batch_size_initial = 2000 #@param {type: "integer"})
  batch_size = 1000 #@param {type: "integer"})
  eval_batch_size = 1000 #@param {type: "integer"}
  train_batch_size = 100 #@param {type: "integer"}
  max_replay_buffer_size = 1000000 #@param {type: "integer"}

  #@markdown network
  n_layers = 2 #@param {type: "integer"}
  size = 64 #@param {type: "integer"}
  learning_rate = 5e-3 #@param {type: "number"}

  #@markdown logging
  video_log_freq = 5 #@param {type: "integer"}
  scalar_log_freq = 1 #@param {type: "integer"}

  #@markdown gpu & run-time settings
  no_gpu = False #@param {type: "boolean"}
  which_gpu = 0 #@param {type: "integer"}
  seed = 1 #@param {type: "integer"}

args = Args()


In [18]:
#@title create directory for logging

if args.do_dagger:
    logdir_prefix = 'q2_'  # The autograder uses the prefix `q2_`
    assert args.n_iter>1, ('DAgger needs more than 1 iteration (n_iter>1) of training, to iteratively query the expert and train (after 1st warmstarting from behavior cloning).')
else:
    logdir_prefix = 'q1_'  # The autograder uses the prefix `q1_`
    assert args.n_iter==1, ('Vanilla behavior cloning collects expert data just once (n_iter=1)')

data_path ='./data'
if not (os.path.exists(data_path)):
    os.makedirs(data_path)
logdir = logdir_prefix + args.exp_name + '_' + args.env_name + \
         '_' + time.strftime("%d-%m-%Y_%H-%M-%S")
logdir = os.path.join(data_path, logdir)
args['logdir'] = logdir
if not(os.path.exists(logdir)):
    os.makedirs(logdir)

In [19]:
## run training
print(args.logdir)
run_training_loop(args)

./data/q2_dagger_bc_ant_Ant-v4_23-08-2025_20-35-01
########################
logging outputs to  ./data/q2_dagger_bc_ant_Ant-v4_23-08-2025_20-35-01
########################
Using GPU id 0


/gpfs/data/oermannlab/users/bml398/.conda/envs/cs_285/lib/python3.11/site-packages/gymnasium/envs/registration.py:512: DeprecationWarning: WARN: The environment Ant-v4 is out of date. You should consider upgrading to version `v5`.
  logger.deprecation(


Loading expert policy from... cs285/policies/experts/Ant.pkl
obs (1, 111) (1, 111)
Done restoring expert policy...


********** Iteration 0 ************


Training agent using sampled data from replay buffer...

Beginning logging procedure...


Eval_AverageReturn : 4539.119140625
Eval_StdReturn : 0.0
Eval_MaxReturn : 4539.119140625
Eval_MinReturn : 4539.119140625
Eval_AverageEpLen : 1000.0
Train_AverageReturn : 4681.891673935816
Train_StdReturn : 30.70862278765526
Train_MaxReturn : 4712.600296723471
Train_MinReturn : 4651.18305114816
Train_AverageEpLen : 1000.0
Training Loss : -16.01653480529785
Train_EnvstepsSoFar : 0
TimeSinceStart : 25.464521169662476
Initial_DataCollection_AverageReturn : 4681.891673935816
Done logging...




********** Iteration 1 ************


Relabelling collected observations with labels from an expert policy...

Training agent using sampled data from replay buffer...

Beginning logging procedure...

Eval_AverageReturn : 4552.107421875
Eval_StdReturn : 0.0
Eva

In [ ]:
#@markdown You can visualize your runs with tensorboard from within the notebook

%load_ext tensorboard
%tensorboard --logdir ./data

## Running DAgger (Problem 2)
Modify the settings above:
1. check the `do_dagger` box
2. set `n_iters` to `10`
3. set `exp_name` to `dagger_{env_name}`
and then rerun the code.